In [108]:
from datetime import datetime
from elasticsearch import Elasticsearch
import nltk
from time import gmtime, strftime
from pythainlp.segment import segment
import codecs
es = Elasticsearch(['localhost'],http_auth=('elastic', 'changeme'),port=9200)


In [19]:
def query_item(id):
    res = es.get(index=index, doc_type=doc_type, id=id)
    #print(res['_source'])
    item=res['_source']
    content=item[0]
    sentiment=item[1]
    return (content,sentiment)

In [20]:
def token_item(content):
    ef=es.indices.analyze(text=content,analyzer="thai")
    gc=[]
    for ge in ef['tokens']:
        gc.append(ge['token'])
    return gc

In [21]:
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
      all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [22]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [27]:
def get_max_id(index):
    res = es.search(index=index, 
                            body={  "stored_fields": [ "_uid"],
                                        "query": {"match_all": {}  },
                                        "size": 1,
                                        "sort": [ { "_uid" : { "order" : "desc"} } ]
                                        })
    return res['hits']['hits'][0]["_id"]

In [110]:
res = es.search(index="logstash-001", body={"query": {"match_all": {}}})
#print("Got %d Hits:" % res['hits']['total'])
ox=[]
for hit in res['hits']['hits']:
    print((hit['_source']['id']))
    #print(hit['_source']['author'])




5
6
7


In [111]:
res

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AVpLO4vO8bU-6iaIcn13',
    '_index': 'logstash-001',
    '_score': 1.0,
    '_source': {'@timestamp': '2017-02-17T08:41:16.592Z',
     '@version': '1',
     'content': 'บริษัท ทิ พยู ไพล์ จำกัด(พม่า) จับมือ บริษัท ฮัจยี กรุ๊ป จำกัด(ไทย) ร่วมลงทุนกว่า 78 โครงการ พัฒนาเมืองเย รัฐมอญ สาธารณรัฐสหภาพเมียนมาร์ หลังรัฐบาลกลางเมียนมาร์ไฟเขียวอนุมัติโครงการ และทุกโครงการพร้อมเดินหน้าลงมือดำเนินการในเร็วๆ นี้',
     'host': '999-PC',
     'id': '5',
     'message': '5,บริษัท ทิ พยู ไพล์ จำกัด(พม่า) จับมือ บริษัท ฮัจยี กรุ๊ป จำกัด(ไทย) ร่วมลงทุนกว่า 78 โครงการ พัฒนาเมืองเย รัฐมอญ สาธารณรัฐสหภาพเมียนมาร์ หลังรัฐบาลกลางเมียนมาร์ไฟเขียวอนุมัติโครงการ และทุกโครงการพร้อมเดินหน้าลงมือดำเนินการในเร็วๆ นี้,strong\r',
     'path': 'D:/search/logstash-5.2.1/bin/industry4.csv',
     'sentiment': 'strong',
     'type': 'industry4'},
    '_type': 'industry4'},
   {'_id': 'AVpLO4vO8bU-6iaIcn14',
    '_index': 'logstash-001',
  

In [85]:
def get_result(idx,idd):
    res = es.search(index=idx, body={  "query": {
        "match" : {"id" : idd } } })
        
    for hit in res['hits']['hits']:
        tk=token_item(hit['_source']['content'])
        st=hit['_source']['sentiment']
        return (tk,st)
    

In [99]:
get_result("logstash-001",5)

(['บริษัท',
  'ทิ',
  'พ',
  'ยู',
  'ไพล์',
  'จำกัด',
  'พม่า',
  'จับ',
  'มือ',
  'บริษัท',
  'ฮัจยี',
  'กรุ๊ป',
  'จำกัด',
  'ไทย',
  'ลงทุน',
  '78',
  'โครงการ',
  'พัฒนา',
  'เมืองเย',
  'รัฐ',
  'มอญ',
  'สาธารณรัฐ',
  'สหภาพ',
  'เมีย',
  'นมาร์',
  'รัฐบาล',
  'กลาง',
  'เมีย',
  'น',
  'มาร์ไฟเขียว',
  'อนุมัติ',
  'โครงการ',
  'โครงการ',
  'เดิน',
  'หน้า',
  'ลงมือ',
  'ดำเนิน',
  'เร็ว',
  'ๆ'],
 'strong')

In [131]:
index="logstash-001"
res = es.search(index, body={"query": {"match_all": {}}})
#print("Got %d Hits:" % res['hits']['total'])
ox=[]
for hit in res['hits']['hits']:
    nid=hit['_source']['id']
    tk=get_result("logstash-001",nid)
    ox.append(tk)
  
    

In [132]:
print(ox)

[(['บริษัท', 'ทิ', 'พ', 'ยู', 'ไพล์', 'จำกัด', 'พม่า', 'จับ', 'มือ', 'บริษัท', 'ฮัจยี', 'กรุ๊ป', 'จำกัด', 'ไทย', 'ลงทุน', '78', 'โครงการ', 'พัฒนา', 'เมืองเย', 'รัฐ', 'มอญ', 'สาธารณรัฐ', 'สหภาพ', 'เมีย', 'นมาร์', 'รัฐบาล', 'กลาง', 'เมีย', 'น', 'มาร์ไฟเขียว', 'อนุมัติ', 'โครงการ', 'โครงการ', 'เดิน', 'หน้า', 'ลงมือ', 'ดำเนิน', 'เร็ว', 'ๆ'], 'strong'), (['รอง', 'นายกรัฐมนตรี', 'แจง', 'เอกชน', 'ชี้', 'เวลา', 'ชาติ', 'เผชิญ', 'ท้าทาย', 'ยก', 'ร่วมมือ', 'เอเชีย', 'สร้าง', 'พลัง', 'ขับเคลื่อน', 'เศรษฐกิจ', 'ใคร', 'เทียบ', 'แนะ', 'ค้น', 'หา', 'จุด', 'แข็ง', 'เชื่อม', 'โยง', 'สู่', 'รุ่งโรจน์', 'ภูมิภาค', 'ลั่น', 'ทำ', 'ไทย', 'ศูนย์', 'กลาง', 'ผลิต', 'อุตสาหกรรม', 'บริการ', 'กระจาย', 'สินค้า', 'ผลิต', 'ยัน', 'รัฐ', 'มุ่ง', 'มั่น', 'ลงทุน', 'โครงสร้าง', 'พื้นฐาน', 'รอง', 'ยก', 'ระดับ', 'เขต', 'เศรษฐกิจ', 'พิเศษ', 'สร้าง', 'ประโยชน์', 'รุ่งเรือง', 'ทุกๆ', 'ชาติ', 'ดูแล', 'ลงทุน', 'ดี'], 'strong'), (['บี', 'โอ', 'ไอ', 'ชะลอ', 'แผนการ', 'อนุมัติ', 'งบประมาณ', 'ลงทุน', 'รอ', 'ชัดเจน'], 'medium'), (['

In [133]:
training_set = nltk.classify.apply_features(extract_features, ox)

In [134]:
word_features = get_word_features(get_words_in_tweets(ox))

In [135]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [116]:
#  ใส่ข้อความตรงอักษรสีแดง ด้านล่างนี้ #

steep= 'คณะรัฐมนตรีอนุมัติงบประมาณลงทุนเขตเศรษฐกิจพิเศษ'

# ระบุ Catagory  : Social ,Technology, Economic, Environment, Polotic #

catatory = "Environment"


steep=steep.replace(",","")
steepx=segment(steep)
#myString = " ".join(steepx)

answer=(classifier.classify(extract_features(myString.split())))
print (steep+" : "+classifier.classify(extract_features(myString.split()))+" signal")

###
#date=strftime("%Y-%m-%d", gmtime())
#text_file = codecs.open("D:/search/logstash-5.0.0/bin/industry4/Output.csv", "a", 'UTF-8')
#text_file.write(date+","+catatory+","+steep+","+str(answer)+"\n")
#text_file.close()
###

คณะรัฐมนตรีอนุมัติงบประมาณลงทุนเขตเศรษฐกิจพิเศษ : strong signal


In [136]:
file =codecs.open("test-data.txt",'r',"utf-8")
for me in file:
    steep=me.replace(",","")
    steepx=segment(me)
    myString = " ".join(steepx)
    print (steep+" : "+classifier.classify(extract_features(myString.split()))+" signal")
    print("")


คณะรัฐมนตรีเห็นชอบโครงการพัฒนาเขตเศรษฐกิจพิเศษ
 : medium signal

บีโอไออนุมัติให้ผู้ประกอบการด้านอุตสาหกรรมรถยนต์ ขยายโรงงานในเขตเศรษฐกิจพิเศษ
 : medium signal

สมาคมธนาคารรอความชัดเจนก่อนผลักดันแผนการลงทุนในระยะ 5 ปีข้างหน้า
 : medium signal

